In [1]:
path = !(echo $(pwd))

In [2]:
import sys
sys.path.insert(0, path[0])

import numpy as np
import chess
import tensorflow
from tensorflow import keras

from utils.board_representation import tensor_to_board
from utils.visualisation import *
from utils.data_loader import load_train_test
from train import sum_squared_loss, pc_column_reg, pc_plane_reg, num_pc_reg, custom_regularized_loss

2023-09-16 12:19:32.776448: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 12:19:32.804874: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 12:19:32.805868: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 12:19:33.582141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
print(tensorflow.__version__)

2.13.0


In [24]:
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

In [25]:
def visualize_tensor(tensor):

    pieces = ["pawn", "knight", "bishop", "rook", "queen", "king"]
    piece_map = ["white "+piece for piece in pieces] + \
        ["black "+piece for piece in pieces] + \
        ["castling rights", "en passant", "turn"]

    def plot_slice(slice_index):
        plt.figure(figsize=(8, 8))
        plt.imshow(tensor[0][:, :, slice_index], cmap='viridis', vmin=0, vmax=1)
        plt.colorbar()
        plt.gca().invert_yaxis()
        plt.title(f"Slice {slice_index} - {piece_map[slice_index]}")
        plt.show()

    return interact(plot_slice, slice_index=IntSlider(min=0, max=tensor[0].shape[2] - 1, step=1, value=0))

In [4]:
train_data, test_data = load_train_test("./data")
print(len(train_data), len(test_data))

90000 10000


In [5]:
model = keras.models.load_model(
    "./model/20230915_trivial_autoencoder.keras",
    custom_objects={
        "sum_squared_loss": sum_squared_loss,
        "num_pc_reg": num_pc_reg,
        "pc_column_reg": pc_column_reg,
        "pc_plane_reg": pc_plane_reg,
        "custom_regularized_loss": custom_regularized_loss
    }
)

In [39]:
index = 2

In [40]:
sample = test_data[index].reshape((1,8,8,15))

In [41]:
reconstruction = model.predict_on_batch(sample)

In [49]:
model.evaluate(x=[reconstruction], y=[sample])

Tensor("custom_regularized_loss/Shape:0", shape=(4,), dtype=int32)
Tensor("Shape_1:0", shape=(4,), dtype=int32)
1/1 [==============================] - 0s 322ms/step - loss: 4.5000 - sum_squared_loss: 2.5879 - num_pc_reg: 1.4150 - pc_column_reg: 2.5879 - pc_plane_reg: 2.3789


[4.5, 2.587890625, 1.4150390625, 2.587890625, 2.37890625]

In [51]:
custom_regularized_loss(sample, reconstruction), sum_squared_loss(sample, reconstruction), num_pc_reg(sample, reconstruction), pc_column_reg(sample, reconstruction), pc_plane_reg(sample, reconstruction)

tf.Tensor([ 1  8  8 15], shape=(4,), dtype=int32)
tf.Tensor([ 1  8  8 15], shape=(4,), dtype=int32)


(<tf.Tensor: shape=(), dtype=float16, numpy=0.705>,
 <tf.Tensor: shape=(), dtype=float16, numpy=0.4854>,
 <tf.Tensor: shape=(), dtype=float16, numpy=0.1334>,
 <tf.Tensor: shape=(), dtype=float16, numpy=0.485>,
 <tf.Tensor: shape=(), dtype=float16, numpy=0.382>)

In [44]:
binary_reconstruction = binarize_array(reconstruction)

In [45]:
sample_board = tensor_to_board(sample[0])
reconstructed_board = tensor_to_board(binary_reconstruction[0])

In [46]:
print_board(sample_board)

k . . . . . . .
. N . . q . . .
. . . . n . . .
. . K . . . . .
. . . . . . . .
. . . . . . . .
. . . q . . . .
. . . . n . . .




In [47]:
print_board(reconstructed_board)

k . . . . . . .
. N . . q . . .
. . . . n . . .
. . K . . . . .
. . . . . . . .
. . . . . . . .
. . . q . . . .
. . . . n . . .




In [48]:
visualize_tensor(abs(reconstruction-sample))

interactive(children=(IntSlider(value=0, description='slice_index', max=14), Output()), _dom_classes=('widget-…

<function __main__.visualize_tensor.<locals>.plot_slice(slice_index)>